In [37]:
from audiolazy.lazy_filters import z, Stream
import wave
import numpy as np
import matplotlib.pyplot as plt
from audiolazy.lazy_analysis import window
import scipy.io.wavfile as wf
from audiolazy.lazy_lpc import acorr
from audiolazy.lazy_lpc import lpc


def read_audio(path):
    source = wf.read(path)
    rate = source[0]
    data = source[1]

    return rate, data


def pre_emphasis(signal):
    start = Stream(signal)
    filter_func = 1 - 0.97 * z ** -1
    output = filter_func(start, zero=0).take(len(signal))
    return np.array(output)


def overlap_analysis_window(signal, window_size, overlap_size):
    start = 0
    stop = len(signal)
    separate_size = window_size - overlap_size
    windows = []
    for start in range(0, len(signal), separate_size):
        end = min(start + window_size, stop)
        windows.append(signal[start:end])
    return np.array(windows)


def hamming_one_window(signal):
    return window.hamming(len(signal)) * signal


def hamming_many_window(signals):
    return np.array([np.array(window.hamming(len(signal))) * signal for signal in signals])


def auto_correlation_many_window(signals, p):
    return np.array([acorr(signal, max_lag=p) for signal in signals])


def save(path, params, frames):
    file = wave.open(path, "w")
    file.setparams(params)
    file.writeframes(frames)
    file.close()


def combine(signals, overlap_size):
    result = np.array([])
    for i in range(0, len(signals)):
        if i > 0:
            result = np.append(result, signals[i][overlap_size:])
        else:
            result = np.append(result, signals[i])

    return result

def lpc_coffiecent(signals, p):
    filter_cfs = np.array([lpc.kautocor(signal, p) for signal in signals])
    return filter_cfs

def analyze(signal, level, overlap_size, window_size):
    print "Create overlap analysis window"
    a = overlap_analysis_window(signal, window_size, overlap_size)
    print "Finish create overlap analysis window"

    print "Find LPC parameters"
    filter_function = np.array([lpc.kautocor(w, level) for w in a])
    print "Finish find LPC parameters"

    output = []
    for index in range(0, len(a)):
        output.append(filter_function[index](a[index], zero=0).take(len(a[index])))

    output = combine(np.array(output), overlap_size)
    return output


def test():
    print "Start"
    level = 30
    overlap_size = 0
    window_size = 500
    rate, channel = read_audio("wav/Xe.wav")
    print len(channel)

    output = analyze(channel, level, overlap_size, window_size)
    wf.write("wav/Xe_out.wav", rate, output)


def visualize(data, rate, name):
    # Extract Raw Audio from Wav File
    time = np.linspace(0, len(data) / rate, num=len(data))
    plt.figure()
    plt.title('Signal Wave...')
    plt.plot(data)
    plt.savefig(name)


In [2]:
from scipy.io import wavfile
file_name = 'Xe.wav'
# file_name = 'A96.wav'
rate, signal = wavfile.read(file_name)

In [3]:
data = signal[0:512]

In [6]:
visualize(data, rate, "voice_{0}".format(0))

In [16]:
a = pre_emphasis(data)

In [17]:
len(a)

512

In [18]:
b = hamming_one_window(a)

In [23]:
type(b)

numpy.ndarray

In [22]:
auto_correlation([b], 20)

array([[ 33332.24056736, -10915.79092679,    554.13806116,   4627.61113924,
         -4241.57278191,   3391.39865938,  -2612.81116688,   5268.27229542,
         -3592.32272168,   3894.75364363,  -2628.36846459,    162.0118101 ,
           438.14483224,   -646.12922826,   3748.23707324,  -1957.62843179,
          5043.28673136,   1152.60566453,    837.00280107,    958.0028183 ,
          1151.93773206]])

In [30]:
signals = overlap_analysis_window(empa, window_size=256, overlap_size=128)

In [39]:
len(cof)

101

In [29]:
empa = pre_emphasis(signal)

In [33]:
ham = hamming_many_window(signals)

In [38]:
cof = lpc_coffiecent(ham, 19)

In [42]:
import matplotlib
matplotlib.use('TkAgg')

from numpy import arange, sin, pi
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

import sys
if sys.version_info[0] < 3:
    import Tkinter as Tk
else:
    import tkinter as Tk


def destroy(e):
    sys.exit()
# def tk_lpc(file_name, root):
root = Tk.Tk()
root.wm_title("LPC")

f = Figure(figsize=(5, 4), dpi=100)
a = f.add_subplot(311)
b = f.add_subplot(312)
# c = f.add_subplot(313)

rate, signal = wavfile.read(file_name)
signal = signal.astype('int64')
a.plot(np.arange(len(signal)), signal)
a.set_title('Speech signal')
a.set_xlabel('samples')
a.set_ylabel('magtitude')

# energy = energy_in_frame(signal)
b.plot(np.arange(len(empa)), empa)
b.set_title('Hieu chinh')
b.set_xlabel('samples')
b.set_ylabel('magtitude')

# c.plot(t, zcr)
# zcr = zero_crossing(signal)
# c.plot(np.arange(len(zcr)), zcr)
# c.set_title('zcr')
# c.set_xlabel('samples')
# c.set_ylabel('zcr')


# a tk.DrawingArea
canvas = FigureCanvasTkAgg(f, master=root)
canvas.show()
canvas.get_tk_widget().pack(side=Tk.TOP, fill=Tk.BOTH, expand=1)

canvas._tkcanvas.pack(side=Tk.TOP, fill=Tk.BOTH, expand=1)

# button = Tk.Button(master=root, text='Quit', command=sys.exit)
button = Tk.Button(master=root, text='Quit', command=root.destroy)
button.pack(side=Tk.BOTTOM)

Tk.mainloop()

In [43]:
p = 19
sizez = 128

In [47]:
out = np.zeros(len(signal))

In [ ]:
cof2 = 

In [46]:
output = []
for index in range(0, len(a)):
        output.append(filter_function[index](a[index], zero=0).take(len(a[index])))

AttributeError: 'ZFilter' object has no attribute 'shape'

In [ ]:
for idx in range(0, len(signal)/sizez):
    start = idx * sizez + p
    stop = start + sizez
    for x in range(start:stop):
        out[x] = 